# 03 - Interactive Viz

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Please read the notebook from [this URL](http://nbviewer.jupyter.org/github/nunomota/ada2017-hw/blob/master/Homework_02/Homework_02.ipynb).

### Content:
* [Task 1](#t1)
* [Task 2](#t2)
* [Task 3](#t3)
* [Task 4](#t4)

# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import folium

# Constants definition

In [ ]:
DATA_PATH = './data/'
TOPOJSON_PATH = './topojson/'
EUROSTAT_FILE = 'lfsq_urgaed_sheets.xls'
AMSTAT_FILE = 'tasso_di_disoccupazione.xlsx'
 
COORDINATE_EUROPE = [54.5260, 15.2551]

# Task 1 <a name="t1">

From the eurostat website we found a dataset that includes the european unemployment rates at a recent date. We chose the dataset containing the quaterly unemployment rates, so that we could get the most recent data. We also made sure that the chosen dataset included the rates for Switzerland as well.

Therefore, we first read the `.xls` file containing the unemployment rates per country. 

In [ ]:
# Read Europe unemployment data
def read_europe_df(sheet):
    # Read xls file
    df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f=EUROSTAT_FILE),
        sheetname=sheet, skiprows=[12,13,14,15,16,17], header=10, na_values=':')

    # Rename country IDs and names columns
    df.rename(columns={'GEO': 'id', 'GEO(L)/TIME': 'name'}, inplace=True)

    # We need to replace the IDs for UK and Greece so that they match TopoJSON IDs
    return df.replace('UK', 'GB').replace('EL', 'GR')

europe_df = read_europe_df('Data5')

We will average the rates for the last 4 quarters, that is:
* 2016, Quarter 3
* 2016, Quarter 4
* 2017, Quarter 1
* 2017, Quarter 2

In this way we have the most recent information over a year, making sure that we avoid any possible seasonal trends on unemployment.

In [ ]:
europe_df['recent'] = europe_df[['2016Q3', '2016Q4', '2017Q1', '2017Q2']].mean(axis=1)
recent_europe_df = europe_df[['id', 'recent']]

In order to create the map of Europe, we will first read and modify the topoJSON file for Europe. We remove from it all the countries that we do not have information about, so that they are not plotted in the map.

In [ ]:
# Load topojson file for Europe
europe_topo_path = r'{tp}europe.topojson.json'.format(tp=TOPOJSON_PATH)

# We keep only the countries we have information of
geo_json = json.load(open(europe_topo_path))
geo_countries = []
for c in geo_json['objects']['europe']['geometries']:
    if c['id'] in list(europe_df['id']):
        geo_countries.append(c)

# We substitute the list of countries
geo_json['objects']['europe']['geometries'] = geo_countries
geo_data = json.dumps(geo_json)

Lastly, we create the map for Europe.

TODO: choose and explain colors

TODO: Choose best background! I plotted all of them to compare them...

TODO: split the intervals into data classes and explain why those

TODO: Add markers including the name of the country and the actual rate? (*interactions you could add in order to make the visualization intuitive and expressive*) (Example of implementation in next cell, maybe put the markers in the capital city of the country?)

TODO: Compare Switzerland's unemployment rate to that of the rest of Europe (it's the main goal of this task)

TODO: Decide how to compute the values for the plot

* (Change and) Explain colors used [url given in assignment](https://carto.com/academy/courses/intermediate-design/choose-colors-1/)
* Export map
* ? Explain intervals considered... [url given in assignment](http://gisgeography.com/choropleth-maps-data-classification/)
* Highlight Switzerland somehow...

Suggestions:
* ? Add Interactivity (eg choose sex, age...)
* ? Set a color scale for countries with less unemployment than switzerland, other for more than Switerland

Useful URLs:
* [Many examples](http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/) (very good)
    * [Markers](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/ContinuousWorld.ipynb), [Markers2](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/FeatureGroup.ipynb), [Marker cluster](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb), [Popups](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Popups.ipynb)
    * [LayerControl and others](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Features.ipynb)
    * [Timestamp (and others?)](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/HeatMapWithTime.ipynb), [Time slider](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb)
    * [Many plugins](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb)
    * [Tiles](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TilesExample.ipynb)
* [Colormap](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/v0.2.0/examples/Colormaps.ipynb), [Colors](https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104)
* [Folium doc](https://media.readthedocs.org/pdf/folium/latest/folium.pdf) (not very good)
* [Pretty example](http://andrewgaidus.com/leaflet_webmaps_python/)

In [ ]:
# Create a new map
map_europe = folium.Map(COORDINATE_EUROPE, tiles=None, zoom_start=3)

# Add tile layers
folium.TileLayer('stamentoner').add_to(map_europe)
folium.TileLayer('OpenStreetMap').add_to(map_europe)
folium.TileLayer('stamenterrain').add_to(map_europe)
folium.TileLayer('cartodbdark_matter').add_to(map_europe)
folium.TileLayer('cartodbpositron').add_to(map_europe)

# Plot total unemployment
map_europe.choropleth(geo_data=json.loads(geo_data), name='Total Population',
    data=recent_europe_df, columns=['id', 'recent'], key_on='feature.id', fill_opacity=1,
    fill_color='YlOrRd', topojson='objects.europe')

# Add markers with information on unemployment per country
#feature_group = FeatureGroup(name='Some icons')
#Marker(location=[45.3288, -121.6625],
       #popup='Mt. Hood Meadows').add_to(feature_group)

#Marker(location=[45.3311, -121.7113],
       #popup='Timberline Lodge').add_to(feature_group)

#feature_group.add_to(m)

# Add LayerControl
folium.LayerControl(collapsed=False).add_to(map_europe)

map_europe

# Task 2 <a name="t2">
Go to the amstat website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

In [ ]:
# Read Swiss unemployment data

# Read xls file
df = pd.read_excel('{dp}{f}'.format(dp=DATA_PATH, f=AMSTAT_FILE), header=[0,1])
df

# Task 3 <a name="t3">
Use the amstat website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between Swiss and foreign workers.

The Economic Secretary (SECO) releases a monthly report on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for foreign (5.1%) and Swiss (2.2%) workers.

Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (hint The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

# Task 4 <a name="t4">

**BONUS**: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in unemployment rates between the areas divided by the Röstigraben?